In [10]:
import io
import fitz
from PIL import Image

In [19]:
# file path you want to extract images from
file = "crvBBSCompact (5).pdf"
# open the file
pdf_file = fitz.open(file)

In [20]:
# iterate over pdf pages
for page_index in range(len(pdf_file)):
    # get the page itself
    page = pdf_file[page_index]
    image_list = page.get_images()
    # printing number of images found in this page
    if image_list:
        print(f"[+] Found a total of {len(image_list)} images in page {page_index}")
    else:
        print("[!] No images found on page", page_index)
    for image_index, img in enumerate(page.get_images(), start=1):
        # get the XREF of the image
        xref = img[0]
        # extract the image bytes
        base_image = pdf_file.extract_image(xref)
        image_bytes = base_image["image"]
        # get the image extension
        image_ext = base_image["ext"]
        # load it to PIL
        image = Image.open(io.BytesIO(image_bytes))
        # save it to local disk
        image.save(open(f"image{page_index+1}_{image_index}.{image_ext}", "wb"))

In [22]:
import matplotlib.pyplot as plt
import keras_ocr
pipeline= keras_ocr.pipeline.Pipeline()
img_path='image1_5.png'
img=keras_ocr.tools.read(img_path)
prediction_groups=pipeline.recognize([img])
keras_ocr.tools.drawAnnotations(image=img,predictions=prediction_groups[0])

In [23]:
import math
import numpy
def midpoint(x1,y1,x2,y2):
    x_mid= int((x1+ x2)/2)
    y_mid= int((y1+ y2)/2)
    return (x_mid, y_mid)

mask=np.zeros(img.shape[:2],dtype='uint8')
for box in prediction_groups[0]:
    #if box[0] in remove_list:
    x0,y0=box[1][0]
    x1,y1=box[1][1]
    x2,y2=box[1][2]
    x3,y3=box[1][3]
    x_mid0, y_mid0= midpoint(x1,y1,x2,y2)
    x_mid1, y_mi1= midpoint(x0,y0,x3,y3)
    thickness= int(math.sqrt((x2- x1) **2 + (y2-y1) **2))
    cv2.line(mask,(x_mid0, y_mid0),(x_mid1, y_mi1),255,thickness)
    img1=cv2.inpaint(img,mask,9,cv2.INPAINT_NS)#NS
    plt.axis('off')
    plt.savefig('file.png')
    plt.imshow(img1)

In [24]:
import numpy as np
import cv2
def find_centroids(dst):
    ret, dst = cv2.threshold(dst, 0.01 * dst.max(), 255, 0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 
                0.001)
    corners = cv2.cornerSubPix(gray,np.float32(centroids[1:]),(5,5), 
              (-1,-1),criteria)
    return corners

image = img1
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

gray = np.float32(gray)

dst = cv2.cornerHarris(gray, 3, 3, 0.04)

dst = cv2.dilate(dst, None)

# Threshold for an optimal value, it may vary depending on the image.
# image[dst > 0.01*dst.max()] = [0, 0, 255]

# Get coordinates
corners = find_centroids(dst)
# To draw the corners
for corner in corners:
    image[int(corner[1]), int(corner[0])] = [0, 0, 255]
int_corners = np.asarray(corners, dtype = int)
print (len(int_corners))
import matplotlib.pyplot as plt

plt.imshow(gray, cmap="gray")
plt.scatter(int_corners[:, 0], int_corners[:, 1])
